# GAN architectures

<img src="images/GANsnRoses.png" width="150pt"/>

Many GANs in our armory!

## [Denoising Diffusion Probabilistic Models (DDPMs)](https://arxiv.org/abs/2006.11239) (Jun 2020)

Idea: __sampling a simple distribution__ (gaussian) and __moving on a path__ toward the data distribution. 

<img src="images/DDPM_path.png" width="750pt"/>

Coinsider a DAE as a _denoising function_, imagine to iterate the process:

<img src="images/DDPM.png" width="750pt"/>

Consider the denoising process a Markov chain with:

$$\large
p(x_{0..T}) = p(x_T)\prod_{t=0}^T p(x_{t-1}|x_t)
$$

where:

$$\large
p(x_{t-1}|x_t) = \mathcal{N}(x_{t-1}; \mu(x_t,t), \Sigma(x_t,t))
$$

The opposite (diffusion) process is:

$$\large
p(x_{1..T}|x_0) = \prod_{t=1}^T q(x_{t}|x_{t-1})
$$

where for small steps it is assumed that:

$$\large
q(x_{t}|x_{t-1}) = \mathcal{N}(x_{t}; \sqrt{1-\beta_t}x_{t-1}, \beta_t I)
$$

For a variance schedule $\beta_1, \cdots, \beta_T$.

### The loss

Combination of forward and backward steps ($p$ and $q$) in __one denoising step is a VAE__.

Objective, __maximization of the ELBO__:

$$\large
\mathcal{L} = \mathbb{E}_q\left[\log p(x_T) + \sum_{t\geq1}\log\frac{p(x_{t-1}|x_t)}{q(x_t|x_{t-1})}\right]
$$

### The architecture

Based on a [PixelCNN++](https://arxiv.org/abs/1701.05517), that is a double-streamed U-Net:

<img src="images/DDPM_arch.png" width="750pt"/>

__Every time-step represents a single net forward pass.__

(The time-step is encoded using a _transformer sinusoidal position encoding_.)

### Some qualitative results

Examples of denoising/diffusion:

<img src="images/DDPM_examples.png" width="750pt"/>

On the MNIST dataset:

<img src="images/DDPM_mnist.gif" width="500pt"/>

## [Diffusion models Beat GANs](https://arxiv.org/abs/2105.05233) (11 May 2021)

After many improvements to DDPM:

<img src="images/DDPM2_examples.png" width="750pt"/>

### List of main changes

- conditional denoising:
 - classifier incorporated in the model;
 - gradient scaling to better guide to the required class;
- AdaGN:
 - $AdaGN(h,y) = y_s GroupNorm(h) + y_b$;
 - $y = [y_s,y_s]$ as linear projection of $t$ and class embedding;
 - [group normalization](https://arxiv.org/abs/1803.08494) acts on __contiguous groups of channels__;
- adaptive variance schedule via NN;
- [Denoising Diffusion Implicit Models (DDIM)](https://arxiv.org/abs/2010.02502);
- other ablations.

# [Texture Generation with Neural Cellular Automata](https://arxiv.org/abs/2105.07299) (15 May 2021)

As anticipated by [Alan Turing](http://www.marconlab.org/papers/turing.pdf),
patterns emerge from [reaction diffusion systems](https://en.wikipedia.org/wiki/Reaction%E2%80%93diffusion_system).

Consider the diffusion model PDE:

$$\large
\frac{\partial s}{\partial t} = f(s,\nabla_x(s),\nabla^2_x(s))
$$

Consider encoding 
[Conway's game of life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)
like rules in $f$.

Consider encoding this system in a deep neural network:

<img src="images/NCA_arch.svg" width="750pt"/>

where:

- $I_3$ is the 3-channels image;
- $K_x$ and $K_y$ are spatial derivative kernels (Sobel filters have been used);
- $K_{lap}$ is a laplacian filter kernel;
- the stochastic update allows to update stochastically the cells (breaking the all-in-one simmetry).

We get [self organizing textures](https://distill.pub/selforg/2021/textures/).